# <span style='background:DarkOliveGreen'> The Trading Strategy using Moving Averages 9x21   🐍</span>

<strong> Introduction </strong>

The objective of this code is to be able to reduce the work of analysing a large number of stocks.
The idea is that the investor already have selected a list of stocks he wants to analyse, and he will use this code to decide which ones to buy, to hold or to stop ("close the position"). 
He can and should combine Fundamental Analysis and Technical Analysis.

He should use Fundamental Analysis to generate the list of companies he would like to invest on - this part is not developed as a code in this Jupyter NoteBook (JNB), but is briefly discussed below.

<strong> Fundamental Analysis </strong>

The idea is to get a list of companies that are solid and could be interesting to hold for some time.
In the beginning of the code, we provide a list of stocks traded in the Brazilian Market (called B3) based on the IDIV Index.
This index gathers stocks that pay good dividends and have good liquidity.  It's composition can be downloaded at B3 site:
https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/indice-dividendos-idiv-composicao-da-carteira.htm

Therefore, using the IDIV is a good way to select good companies and then we can use Technical Analysis to decide WHEN to: buy, hold or stop.


<strong> Technical Analysis </strong>

The list will be be tested by a classic Technical Indicator: the combination of two Moving Averages (MAs).
In this code I use the MAs of 9 and 21 days, as it is a classic.  But the code can be easily adapted for any other combination of MAs.

<strong> Why using MAs? </strong>

MAs are used to identify trends.  The idea is that if the "fast" one (9 periods) crosses the "slow" one (21 periods) upwards, the stock is in a bullish trend, so we should buy it; 
if it crosses downwards, we should sell it (in this case we will use it as a STOP sign, we should "close the position" (that is, sell all the stocks we have).
We could also open a short position on that specific stock, however we are not this risk-taker and, due to liquidity issues in the Brazilian market, that might be not a good idea.


Hence, the idea is that this code should be run daily. </p>
It will give you the list of stocks classified in 4 cases:
1. if MA9 crossed MA21 up on the previous day: buy
2. if MA9 is higher than MA21 but it was already higher on the day before: hold (if you already bought it before, otherwise wait)
3. if MA9 crossed MA21 down: sell (if you already bought it before, otherwise wait)
obs: If we sell above the price we bought, it is considered "Take Profit" (TP), if we sell below the price we paid, it is considered "Stop Loss" (SL).
4. if MA9 is lower than MA21 and it was already lower on the day before: wait (you should not have a stock in this classification in your portfolio)
(for more on this topic: https://www.daytradetheworld.com/trading-blog/alexander-elder-trade-market-beginners-2/#5_-_Create_a_money_Management_Plan)

As we can see, the movement of crossing is used as signal for buying or selling.
However, I think it should not be used alone, you should combine it with a few more technical indicators.
I use this code to reduce the number of stocks I will analyse - I open a full Graph of the stocks in situation i) and check a few other indicators, as the RSI, for example (I like to use TradingView website for it).  Then I decide to buy it or not.

However, it is strongly advised to use iii) as a STOP signal!

Stocks in situation ii) are more complicated - you can chose to use another criteria to make a partial TP.  That could be a RSI > 70, for example.
Stocks in situation iv): if you have time you can check on them, maybe one of them is "giving" a signal of reversing the tendency (for example: a hammer candle) that the criteria MA9x21 will not identify.  However, if you buy a stock classified on "wait" it is a riskier trade and you should define another criteria for SL - as MA9 is already below MA21 this code will not give you a SL signal.

In [3]:
# importing libraries we'll need to get and treat data
import yfinance as yf
import pandas as pd

In [4]:
# creating the list of tickers: IDIV index as of 23/mar/22
idiv = ['ABCB4.SA', 'BRSR6.SA', 'BBSE3.SA', 'BBDC3.SA', 'BBDC4.SA', 'BRAP4.SA', 'BBAS3.SA', 'AGRO3.SA', 'CCRO3.SA', 'CMIG3.SA', 'CMIG4.SA', 'CESP6.SA', 'CSMG3.SA', 'CPLE3.SA', 'CPLE6.SA', 'CPFE3.SA', 'CYRE3.SA', 'DIRR3.SA', 'ELET3.SA', 'ELET6.SA', 'ENAT3.SA', 'ENBR3.SA', 'EGIE3.SA', 'ROMI3.SA', 'MYPK3.SA', 'ITSA4.SA', 'ITUB3.SA', 'ITUB4.SA', 'JHSF3.SA', 'MRVE3.SA', 'PSSA3.SA', 'QUAL3.SA', 'SAPR4.SA', 'SANB11.SA', 'CSNA3.SA', 'SYNE3.SA', 'TAEE11.SA', 'TGMA3.SA', 'VIVT3.SA', 'TRPL4.SA', 'UNIP6.SA', 'WIZS3.SA']

In [5]:
# getting the quotes for the list and saving it as a Pandas dataframe (df)
# we will get just the "Adj Close" because that is the one used by Technical Indicators / to build Graphs
dfidiv = yf.download(idiv, start="2022-01-01", end = "2022-03-22")["Adj Close"]
# checking the df
dfidiv.head()

[*********************100%***********************]  42 of 42 completed


,ABCB4.SA,AGRO3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BRAP4.SA,BRSR6.SA,CCRO3.SA,CESP6.SA,...,ROMI3.SA,SANB11.SA,SAPR4.SA,SYNE3.SA,TAEE11.SA,TGMA3.SA,TRPL4.SA,UNIP6.SA,VIVT3.SA,WIZS3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,15.55,28.360001,28.264404,16.500881,19.636387,19.813507,24.650000,9.202440,11.44,20.889999,...,16.920689,29.763933,3.68,7.26,36.480000,14.74,24.230000,101.699997,47.707657,7.66
2022-01-04,15.74,27.400000,28.293827,16.577982,19.765133,19.765415,24.879999,9.289621,11.43,20.370001,...,15.979017,30.192051,3.65,7.30,36.660000,13.97,24.260000,101.959999,47.707657,7.37
2022-01-05,15.02,27.490000,27.823082,16.468193,19.625380,19.524960,24.940001,9.124947,11.26,19.990000,...,15.684743,29.958532,3.61,6.93,36.290001,13.31,23.900000,96.430000,47.647785,7.10
2022-01-06,15.25,28.270000,28.048647,16.647846,19.904888,19.659616,25.650000,9.183066,11.01,19.670000,...,15.106006,30.201782,3.51,6.70,36.209999,13.03,23.790001,94.410004,46.879433,7.12
2022-01-07,15.36,28.059999,28.078068,16.717712,20.194376,19.044050,26.660000,9.134633,11.26,19.750000,...,15.164861,30.299080,3.56,6.76,36.240002,13.06,23.660000,95.449997,46.560116,7.14


In [6]:
# checking if we got all quotes for all the tickers on the list
print(len(idiv))
dfidiv.shape

42


(56, 42)

In [7]:
# ok! 42 tickers on the list and 45 columns on the df

In [17]:
dfidiv.tail()

,ABCB4.SA,AGRO3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BRAP4.SA,BRSR6.SA,CCRO3.SA,CESP6.SA,...,ROMI3.SA,SANB11.SA,SAPR4.SA,SYNE3.SA,TAEE11.SA,TGMA3.SA,TRPL4.SA,UNIP6.SA,VIVT3.SA,WIZS3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2022-03-16,16.230000,32.090000,33.480000,17.410000,21.100000,22.799999,33.580002,10.326107,12.36,22.100000,...,12.528887,34.150002,3.97,6.24,41.349998,15.02,24.730000,104.760002,51.490002,7.62
2022-03-17,16.400000,33.200001,33.790001,17.540001,21.180000,23.299999,34.619999,10.310000,12.78,22.969999,...,13.170635,34.750000,3.99,6.45,41.750000,14.85,25.049999,106.400002,52.250000,7.80
2022-03-18,16.879999,33.500000,33.720001,17.500000,21.129999,23.340000,35.209999,10.300000,12.85,23.510000,...,13.555683,34.980000,3.97,6.76,42.419998,15.54,25.370001,107.400002,53.049999,8.16
2022-03-21,16.700001,33.730000,34.410000,17.830000,21.629999,23.639999,36.439999,10.490000,12.81,23.600000,...,13.220000,35.919998,3.99,6.90,42.430000,14.92,25.719999,107.559998,53.369999,8.38
2022-03-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# deleting the null lines
dfidiv = dfidiv.dropna(axis = 0, how ='any')
# and visualizing the last lines
dfidiv.tail()

,ABCB4.SA,AGRO3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BRAP4.SA,BRSR6.SA,CCRO3.SA,CESP6.SA,...,ROMI3.SA,SANB11.SA,SAPR4.SA,SYNE3.SA,TAEE11.SA,TGMA3.SA,TRPL4.SA,UNIP6.SA,VIVT3.SA,WIZS3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2022-03-15,15.720000,32.080002,33.040001,17.120001,20.799999,22.459999,32.810001,10.045190,11.78,22.500000,...,12.094474,33.290001,3.92,6.40,41.009998,14.52,24.590000,101.430000,50.869999,7.23
2022-03-16,16.230000,32.090000,33.480000,17.410000,21.100000,22.799999,33.580002,10.326107,12.36,22.100000,...,12.528887,34.150002,3.97,6.24,41.349998,15.02,24.730000,104.760002,51.490002,7.62
2022-03-17,16.400000,33.200001,33.790001,17.540001,21.180000,23.299999,34.619999,10.310000,12.78,22.969999,...,13.170635,34.750000,3.99,6.45,41.750000,14.85,25.049999,106.400002,52.250000,7.80
2022-03-18,16.879999,33.500000,33.720001,17.500000,21.129999,23.340000,35.209999,10.300000,12.85,23.510000,...,13.555683,34.980000,3.97,6.76,42.419998,15.54,25.370001,107.400002,53.049999,8.16
2022-03-21,16.700001,33.730000,34.410000,17.830000,21.629999,23.639999,36.439999,10.490000,12.81,23.600000,...,13.220000,35.919998,3.99,6.90,42.430000,14.92,25.719999,107.559998,53.369999,8.38


In [19]:
dfidiv.shape

(54, 42)

In [20]:
# so we cut 2 lines, before it was 56 lines...

In [21]:
# we'll make a copy of this df in order to keep the original if we need to check anything...
# the copy will be used to calculate the MAs 9 and 21
dfidivb = dfidiv.copy()
dfidivb.head()

,ABCB4.SA,AGRO3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BRAP4.SA,BRSR6.SA,CCRO3.SA,CESP6.SA,...,ROMI3.SA,SANB11.SA,SAPR4.SA,SYNE3.SA,TAEE11.SA,TGMA3.SA,TRPL4.SA,UNIP6.SA,VIVT3.SA,WIZS3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,15.55,28.360001,28.264404,16.500881,19.636387,19.813507,24.650000,9.202440,11.44,20.889999,...,16.920689,29.763933,3.68,7.26,36.480000,14.74,24.230000,101.699997,47.707657,7.66
2022-01-04,15.74,27.400000,28.293827,16.577982,19.765133,19.765415,24.879999,9.289621,11.43,20.370001,...,15.979017,30.192051,3.65,7.30,36.660000,13.97,24.260000,101.959999,47.707657,7.37
2022-01-05,15.02,27.490000,27.823082,16.468193,19.625380,19.524960,24.940001,9.124947,11.26,19.990000,...,15.684743,29.958532,3.61,6.93,36.290001,13.31,23.900000,96.430000,47.647785,7.10
2022-01-06,15.25,28.270000,28.048647,16.647846,19.904888,19.659616,25.650000,9.183066,11.01,19.670000,...,15.106006,30.201782,3.51,6.70,36.209999,13.03,23.790001,94.410004,46.879433,7.12
2022-01-07,15.36,28.059999,28.078068,16.717712,20.194376,19.044050,26.660000,9.134633,11.26,19.750000,...,15.164861,30.299080,3.56,6.76,36.240002,13.06,23.660000,95.449997,46.560116,7.14


In [22]:
dfidivb.tail()

,ABCB4.SA,AGRO3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BRAP4.SA,BRSR6.SA,CCRO3.SA,CESP6.SA,...,ROMI3.SA,SANB11.SA,SAPR4.SA,SYNE3.SA,TAEE11.SA,TGMA3.SA,TRPL4.SA,UNIP6.SA,VIVT3.SA,WIZS3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2022-03-15,15.720000,32.080002,33.040001,17.120001,20.799999,22.459999,32.810001,10.045190,11.78,22.500000,...,12.094474,33.290001,3.92,6.40,41.009998,14.52,24.590000,101.430000,50.869999,7.23
2022-03-16,16.230000,32.090000,33.480000,17.410000,21.100000,22.799999,33.580002,10.326107,12.36,22.100000,...,12.528887,34.150002,3.97,6.24,41.349998,15.02,24.730000,104.760002,51.490002,7.62
2022-03-17,16.400000,33.200001,33.790001,17.540001,21.180000,23.299999,34.619999,10.310000,12.78,22.969999,...,13.170635,34.750000,3.99,6.45,41.750000,14.85,25.049999,106.400002,52.250000,7.80
2022-03-18,16.879999,33.500000,33.720001,17.500000,21.129999,23.340000,35.209999,10.300000,12.85,23.510000,...,13.555683,34.980000,3.97,6.76,42.419998,15.54,25.370001,107.400002,53.049999,8.16
2022-03-21,16.700001,33.730000,34.410000,17.830000,21.629999,23.639999,36.439999,10.490000,12.81,23.600000,...,13.220000,35.919998,3.99,6.90,42.430000,14.92,25.719999,107.559998,53.369999,8.38


In [23]:
# In this step we will calculate de Mas 9 and 21 and their difference (MA09 minus MA21), and will 
# replace the value of every cell with this difference (this will generate null values for the first 21 lines)
for (columnName) in dfidivb:
    dfidivb[columnName] = (dfidivb[columnName].rolling(9).mean()) - (dfidivb[columnName].rolling(21).mean())

In [24]:
# having a look at this new df
dfidivb.head(25)

,ABCB4.SA,AGRO3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BRAP4.SA,BRSR6.SA,CCRO3.SA,CESP6.SA,...,ROMI3.SA,SANB11.SA,SAPR4.SA,SYNE3.SA,TAEE11.SA,TGMA3.SA,TRPL4.SA,UNIP6.SA,VIVT3.SA,WIZS3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
dfidivb.tail()

,ABCB4.SA,AGRO3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BRAP4.SA,BRSR6.SA,CCRO3.SA,CESP6.SA,...,ROMI3.SA,SANB11.SA,SAPR4.SA,SYNE3.SA,TAEE11.SA,TGMA3.SA,TRPL4.SA,UNIP6.SA,VIVT3.SA,WIZS3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2022-03-15,-0.312381,1.452222,-0.554421,-0.326853,-0.291391,-0.367302,2.353016,-0.071652,-0.449048,0.005556,...,-1.041997,0.002857,0.010952,0.046508,1.261747,0.073175,0.318889,1.667460,-0.014605,-0.334127
2022-03-16,-0.235714,1.125714,-0.708029,-0.289761,-0.224869,-0.399207,2.329683,-0.034903,-0.367619,-0.051111,...,-1.069266,0.226826,-0.003016,0.022381,1.250000,0.047619,0.262540,2.053809,0.201743,-0.272222
2022-03-17,-0.122064,0.879048,-0.739355,-0.201564,-0.086285,-0.311429,2.180794,-0.003790,-0.224762,-0.057460,...,-1.008931,0.590000,-0.017937,-0.049524,1.106508,-0.006032,0.167302,2.568095,0.482537,-0.194921
2022-03-18,0.054603,0.545715,-0.540768,-0.048614,0.106742,-0.183175,1.877302,0.097109,-0.027778,-0.026826,...,-0.811940,1.051746,-0.016825,-0.066984,1.096349,0.037778,0.163333,3.309048,0.774685,-0.041905
2022-03-21,0.190635,0.468254,-0.316636,0.116868,0.315632,-0.014286,1.762857,0.163944,0.103175,0.010635,...,-0.644569,1.578730,-0.008889,-0.049048,1.121905,0.038095,0.196190,4.051429,1.083152,0.123175


In [26]:
dfidivb.shape

(54, 42)

In [29]:
# deleting the null lines
dfidivb = dfidivb.dropna(axis = 0, how ='any')
# and visualizing the df
dfidivb.shape

(34, 42)

In [30]:
# Creating the messages that will be displayed for the tickers (regarding the 4 cases explained in the introduction of this JNB)
# The messages will be based in the values in the last 2 lines of the df (if we call the day you are checking d, they will be d-1 and d-2)
# The expresison: "d-1 > 0" will be used to represent the value on d-1 is positive, etc.
# i) if d-1 > 0  and  d-2 < 0 : BUY
# ii) if d-1 < 0  and  d-2 <0 : STOP
# iii) if d-1 > 0  and  d-2 > 0 : HOLD
# iv) if d-1 < 0  and  d-2 < 0 : WAIT

In [31]:
# creating functions for colored messages to be used
def prRed(skk): print("\033[91m {}\033[00m" .format(skk))
def prGreen(skk): print("\033[92m {}\033[00m" .format(skk))
def prYellow(skk): print("\033[93m {}\033[00m" .format(skk))
def prCyan(skk): print("\033[96m {}\033[00m" .format(skk))
def prLightGray(skk): print("\033[97m {}\033[00m" .format(skk))

In [32]:
# creating the function to classify the tickers and print the messages
def alertacolor (acao):
    if (dfidivb.at[dfidivb.index[-1],acao] > 0) and (dfidivb.at[dfidivb.index[-2],acao] < 0):
        prGreen('  Buy!  : ) ')
    elif (dfidivb.at[dfidivb.index[-1],acao] < 0) and (dfidivb.at[dfidivb.index[-2],acao] > 0):
        prRed('  Stoooop!!')
    elif (dfidivb.at[dfidivb.index[-1],acao] > 0) and (dfidivb.at[dfidivb.index[-2],acao] > 0):
        prGreen('  Keep Calm and... Hold  : )')
    else:
        prYellow('  Wait...')

In [33]:
# running this function for the list of tickers
for papel in idiv:
    prCyan (papel+': ') 
    alertacolor(papel)
    print(' ')

 ABCB4.SA: 
   Keep Calm and... Hold  : )
 
 BRSR6.SA: 
   Keep Calm and... Hold  : )
 
 BBSE3.SA: 
   Wait...
 
 BBDC3.SA: 
   Buy!  : ) 
 
 BBDC4.SA: 
   Keep Calm and... Hold  : )
 
 BRAP4.SA: 
   Keep Calm and... Hold  : )
 
 BBAS3.SA: 
   Wait...
 
 AGRO3.SA: 
   Keep Calm and... Hold  : )
 
 CCRO3.SA: 
   Buy!  : ) 
 
 CMIG3.SA: 
   Keep Calm and... Hold  : )
 
 CMIG4.SA: 
   Keep Calm and... Hold  : )
 
 CESP6.SA: 
   Buy!  : ) 
 
 CSMG3.SA: 
   Wait...
 
 CPLE3.SA: 
   Keep Calm and... Hold  : )
 
 CPLE6.SA: 
   Keep Calm and... Hold  : )
 
 CPFE3.SA: 
   Keep Calm and... Hold  : )
 
 CYRE3.SA: 
   Wait...
 
 DIRR3.SA: 
   Wait...
 
 ELET3.SA: 
   Wait...
 
 ELET6.SA: 
   Wait...
 
 ENAT3.SA: 
   Keep Calm and... Hold  : )
 
 ENBR3.SA: 
   Keep Calm and... Hold  : )
 
 EGIE3.SA: 
   Keep Calm and... Hold  : )
 
 ROMI3.SA: 
   Wait...
 
 MYPK3.SA: 
   Wait...
 
 ITSA4.SA: 
   Buy!  : ) 
 
 ITUB3.SA: 
   Buy!  : ) 
 
 ITUB4.SA: 
   Buy!  : ) 
 
 JHSF3.SA: 
   Wait...
 
 MRVE3.SA:

<div class="alert-info">
    ok!
</div>

<div class="alert-warning">
</br>
  <strong> References </strong>
</div>

###### Youtube Video (in Portuguese): "PYTHON PARA INVESTIMENTOS #4: Simulando uma CARTEIRA DE AÇÕES e comparando com o IBOVESPA"
https://youtu.be/TiNLwmLN-iE?list=PLCAhGm8nJ9CBn51o0x3j1p1LuMRqpeqCy

https://www.geeksforgeeks.org/print-colors-python-terminal/

###### last update: 23.mar.22

###### to go back to my GitHub Page:  https://rafsz.github.io/